In [35]:
import pandas as pd
# # load your data in dataframe df here
# d = {'A':[0.161729, 0.862661, 0.814303, 1.898132, 2.124829], 'B':[0.814335, 0.517964, 0.337391, 1.530963, 0.289176], 'C':[1.2 ,2.3, 5.2, 2.1, 5.2]}
# df = pd.DataFrame(data=d)
# df.head()
# #        A         B      C
# # 0.161729  0.814335    foo
# # 0.862661  0.517964    foo
# # 0.814303  0.337391    foo
# # 1.898132  1.530963    bar
# # 2.124829  0.289176    bar

In [36]:
df = pd.read_csv('X:\RA_AirQuality\combined_final_dataset.csv')

In [37]:
df=df[pd.isnull(df['PM2.5'])==False]

In [38]:
df=df.fillna(df.mean())

In [39]:
df.isnull().values.any()

False

In [40]:
df.shape

(7273, 14)

In [41]:
df.head()

,DateTime,TEMP,CO,NO,NO2,PM10,PM2.5,HUM,DIR,SPD,PRES,Month,Hour,TF_CNT
0,2019-01-01 00:00:00,-13.26,0.28,1.66,7.00,8.62,6.30,80.04,189.90,2.65,625.00,1,0,2042.0
1,2019-01-01 01:00:00,-14.22,0.27,0.66,6.00,8.71,4.41,75.21,282.43,2.70,625.24,1,1,1949.0
2,2019-01-01 02:00:00,-14.30,0.29,0.63,7.31,6.82,5.24,79.09,310.47,2.26,630.04,1,2,1527.0
3,2019-01-01 03:00:00,-14.44,0.29,0.56,6.42,9.10,6.56,79.54,295.01,2.18,630.40,1,3,1403.0
4,2019-01-01 04:00:00,-14.82,0.29,0.72,8.13,5.94,4.70,79.86,298.92,1.55,630.40,1,4,1793.0


In [42]:
df['PM2.5_Round_Up']=df['PM2.5'].astype(int)

In [43]:
# (pm2.5 > 6) = 1
# (pm2.5 <= 6 and pm2.5 >= 3) = 0
# (pm2.5 < 3) = -1
df['PM2.5_cat']=df['PM2.5'].apply(lambda x: 1 if x>6 else 0 if (x<=6 and x>=3) else -1)

In [44]:
df['PM2.5_cat'].value_counts()

-1    2735
 0    2400
 1    2138
Name: PM2.5_cat, dtype: int64

In [45]:
# Uncomment for Approach 1 (Gaussian + Categorical)
# Bin continuous vairbales into 20% quantiles
df['TF_CNT_qt'] = pd.qcut(df['TF_CNT'], 5, labels=['bottom 20', 'lower 20', 'middle 20', 'upper 20', 'top 20'])
df['SPD_qt'] = pd.qcut(df['SPD'], 5, labels=['bottom 20', 'lower 20', 'middle 20', 'upper 20', 'top 20'])
df['DIR_qt'] = pd.qcut(df['DIR'], 5, labels=['bottom 20', 'lower 20', 'middle 20', 'upper 20', 'top 20'])
df['NO2_qt'] = pd.qcut(df['NO2'], 5, labels=['bottom 20', 'lower 20', 'middle 20', 'upper 20', 'top 20'])
df['TEMP_qt'] = pd.qcut(df['TEMP'], 5, labels=['bottom 20', 'lower 20', 'middle 20', 'upper 20', 'top 20'])

In [85]:
# Function that handles sample splitting, model fitting and report printing 
def mfunc(X, y, typ):
    
    # Create training and testing samples
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    # Fit the model
    model = typ
    clf = model.fit(X_train, y_train)

    # Predict class labels on a test data
    pred_labels = model.predict(X_test)

    # Print model attributes 
    print('Classes: ', clf.classes_) # class labels known to the classifier
    if str(typ)=='GaussianNB()':
        print('Class Priors: ',clf.class_prior_) # prior probability of each class.
    else: 
        print('Class Log Priors: ',clf.class_log_prior_) # log prior probability of each class.
        
    # Use score method to get accuracy of the model
    print('--------------------------------------------------------')
    score = model.score(X_test, y_test)
    print('Accuracy Score: ', score)
    print('--------------------------------------------------------')
    
    # Look at classification report to evaluate the model
    print(classification_report(y_test, pred_labels))
    
    # Probabilities of each class for the test data
    print(model.predict_proba(X_test))
    
    print("X_test Shape: ", X_test.shape)
    print(X_test)
    
    print("y_test Shape: ", y_test.shape)    
    print(y_test)
    
    # Return relevant data for chart plotting
    return X_train, X_test, y_train, y_test, clf, pred_labels

In [86]:
import numpy as np # for data manipulation

from sklearn.model_selection import train_test_split # for splitting the data into train and test samples
from sklearn.metrics import classification_report # for model evaluation metrics
from sklearn.preprocessing import OrdinalEncoder # for encoding categorical features from strings to number arrays

import plotly.express as px  # for data visualization
import plotly.graph_objects as go # for data visualization

# Differnt types of Naive Bayes Classifiers
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import BernoulliNB

In [87]:
# (pm2.5 > 6) = 1
# (pm2.5 <= 6 and pm2.5 >= 3) = 0
# (pm2.5 < 3) = -1

# Select data for modeling
X=df[['TF_CNT','TEMP', 'Month', 'Hour']]
y=df['PM2.5_cat'].values

# Fit the model and print the result
X_train, X_test, y_train, y_test, clf, pred_labels, = mfunc(X, y, GaussianNB())

Classes:  [-1  0  1]
Class Priors:  [0.3774493  0.32605706 0.29649364]
--------------------------------------------------------
Accuracy Score:  0.50446735395189
--------------------------------------------------------
              precision    recall  f1-score   support

          -1       0.56      0.50      0.53       539
           0       0.46      0.38      0.41       503
           1       0.49      0.67      0.56       413

    accuracy                           0.50      1455
   macro avg       0.50      0.51      0.50      1455
weighted avg       0.51      0.50      0.50      1455

[[0.2020553  0.18187301 0.61607169]
 [0.55112635 0.40657968 0.04229397]
 [0.34951986 0.42493343 0.2255467 ]
 ...
 [0.71462662 0.18575046 0.09962292]
 [0.49598535 0.43983974 0.0641749 ]
 [0.1815384  0.22157035 0.59689125]]
X_test Shape:  (1455, 4)
           TF_CNT   TEMP  Month  Hour
2173  7979.000000  -0.20      4    13
5780  4934.151461  32.20      8    20
4170  6292.000000  16.77      6    18
5

In [49]:
# CATEGORICAL + GAUSSIAN (APPROACH 1)
# Select data for modeling
X=df[['TF_CNT_qt','TEMP_qt', 'Month', 'Hour']]
y=df['PM2.5_cat'].values

# Encode categorical variables
enc = OrdinalEncoder()
X = enc.fit_transform(X)

# Fit the model and print the result
X_train, X_test, y_train, y_test, clf, pred_labels, = mfunc(X, y, CategoricalNB())

from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
MSE = mean_squared_error(y_test, pred_labels)
MAE = mean_absolute_error(y_test, pred_labels)
RMSE = math.sqrt(MSE)
print("Root Mean Square Error:", RMSE)
print("Mean Absolute Error:", MAE)
print("Mean Squared Error:", MSE)

Classes:  [-1  0  1]
Class Log Priors:  [-0.97431904 -1.12068287 -1.21572951]
--------------------------------------------------------
Accuracy Score:  0.5223367697594502
--------------------------------------------------------
              precision    recall  f1-score   support

          -1       0.58      0.54      0.56       539
           0       0.46      0.44      0.45       503
           1       0.52      0.60      0.55       413

    accuracy                           0.52      1455
   macro avg       0.52      0.53      0.52      1455
weighted avg       0.52      0.52      0.52      1455

Root Mean Square Error: 0.8963461530663467
Mean Absolute Error: 0.5862542955326461
Mean Squared Error: 0.8034364261168385


In [30]:
# CATEGORICAL + GAUSSIAN (APPROACH 2)
# ----- Prepare data -----
# Select data for modeling
X_G=df[['TF_CNT', 'TEMP']] # Gaussian, i.e. continuous
X_C=df[['Month', 'Hour']] # Categorical, i.e. discrete
y=df['PM2.5_cat'].values

# Encode categorical variables
enc = OrdinalEncoder()
X_C = enc.fit_transform(X_C)

# Combine all four variables into one array
X=np.c_[X_G, X_C[:,0].ravel(), X_C[:,1].ravel()]

# Create training and testing samples
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# ----- Fit the two models -----
# Now use the Gaussian model for continuous independent variable and 
model_G = GaussianNB()
clf_G = model_G.fit(X_train[:,0:2], y_train)
# Categorical model for discrete independent variable
model_C = CategoricalNB()
clf_C = model_C.fit(X_train[:,2:4], y_train)


# ----- Get probability predictions from each model -----
# On training data
G_train_probas = model_G.predict_proba(X_train[:,0:2])
C_train_probas = model_C.predict_proba(X_train[:,2:4])
# And on testing data
G_test_probas = model_G.predict_proba(X_test[:,0:2])
C_test_probas = model_C.predict_proba(X_test[:,2:4])

# Combine probability prediction for class=1 from both models into a 2D array
X_new_train = np.c_[(G_train_probas[:,1], C_train_probas[:,1])] # Train
X_new_test = np.c_[(G_test_probas[:,1], C_test_probas[:,1])] # Test


# ----- Fit Gaussian model on the X_new -----
model = GaussianNB()
clf = model.fit(X_new_train, y_train)

# Predict class labels on a test data
pred_labels = model.predict(X_new_test)



# ----- Print results -----
print('Classes: ', clf.classes_) # class labels known to the classifier
print('Class Priors: ',clf.class_prior_) # probability of each class.
# Use score method to get accuracy of model
print('--------------------------------------------------------')
score = model.score(X_new_test, y_test)
print('Accuracy Score: ', score)
print('--------------------------------------------------------')
# Look at classification report to evaluate the model
print(classification_report(y_test, pred_labels))



# Probabilities of each class for the test data
print(model.predict_proba(X_new_test))

Classes:  [-1  0  1]
Class Priors:  [0.3774493  0.32605706 0.29649364]
--------------------------------------------------------
Accuracy Score:  0.4323024054982818
--------------------------------------------------------
              precision    recall  f1-score   support

          -1       0.40      0.32      0.36       539
           0       0.46      0.61      0.52       503
           1       0.42      0.37      0.39       413

    accuracy                           0.43      1455
   macro avg       0.43      0.43      0.42      1455
weighted avg       0.43      0.43      0.42      1455

[[0.35902912 0.1696102  0.47136068]
 [0.28300703 0.62488074 0.09211223]
 [0.31483924 0.55671335 0.12844741]
 ...
 [0.28546081 0.04610854 0.66843065]
 [0.30218944 0.58858212 0.10922844]
 [0.445108   0.15435508 0.40053692]]


In [31]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
MSE = mean_squared_error(y_test, pred_labels)
MAE = mean_absolute_error(y_test, pred_labels)
RMSE = math.sqrt(MSE)
print("Root Mean Square Error:", RMSE)
print("Mean Absolute Error:", MAE)
print("Mean Squared Error:", MSE)

Root Mean Square Error: 1.0574559368688816
Mean Absolute Error: 0.7512027491408935
Mean Squared Error: 1.118213058419244


In [18]:
# from sklearn.naive_bayes import GaussianNB
# clf = GaussianNB()
# X, y = df[['TF_CNT','TEMP']], df['PM2.5_Round_Up']

# # fit the classifier on the training dataset
# clf.fit(X, y)

In [19]:
# # predict the Pr(C = 'bar' | A, B) with predict_proba() 
# print(clf.predict_proba([[1,1]])[:,0])   # Pr(C='bar'|A=1.0, B=1.0)
# # [ 0.86871233]

In [20]:
# # Specify a size of the mesh to be used
# mesh_size = 5
# margin = 1

# # Create a mesh grid on which we will run our model
# x_min, x_max = X.iloc[:, 0].fillna(X.mean()).min() - margin, X.iloc[:, 0].fillna(X.mean()).max() + margin
# y_min, y_max = X.iloc[:, 1].fillna(X.mean()).min() - margin, X.iloc[:, 1].fillna(X.mean()).max() + margin
# xrange = np.arange(x_min, x_max, mesh_size)
# yrange = np.arange(y_min, y_max, mesh_size)
# xx, yy = np.meshgrid(xrange, yrange)

# # Create classifier, run predictions on grid
# Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
# Z = Z.reshape(xx.shape)

# # Specify traces 
# trace_specs = [
#     #[X_train, y_train, 0, 'Train', 'brown'],
#     #[X_train, y_train, 1, 'Train', 'aqua'],
#     [X_test, y_test, 0, 'Test', 'red'],
#     [X_test, y_test, 1, 'Test', 'blue'],
#     [X_test, y_test, -1, 'Test', 'aqua']
# ]

# # Build the graph using trace_specs from above
# fig = go.Figure(data=[
#     go.Scatter(
#         x=X[y==label].iloc[:, 0], y=X[y==label].iloc[:, 1],
#         name=f'{split} data, Actual Class: {label}',
#         mode='markers', marker_color=marker
#     )
#     for X, y, label, split, marker in trace_specs
# ])

# # Update marker size
# fig.update_traces(marker_size=3, marker_line_width=0)

# # Update axis range
# fig.update_xaxes(range=[0, 14000])
# fig.update_yaxes(range=[-50,50])

# # Update chart title and legend placement
# fig.update_layout(title_text="Decision Boundary", 
#     legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

# # Add contour graph
# fig.add_trace(
#     go.Contour(
#         x=xrange,
#         y=yrange,
#         z=Z,
#         showscale=True,
#         colorscale='magma',
#         opacity=1,
#         name='Score',
#         hoverinfo='skip'
#     )
# )

# fig.show()

In [21]:
# X-axis = Traffic count
# Y-axis = Temp
# Range of color represents the probabilities of 'pm2.5>10' (class = 1)
# Blue = pm2.5>10 (class 1)
# Red = pm2.5<=10 (class 0)

In [34]:
# print(model.predict_proba(X_new_test))
print(X_new_test)

[[0.20477056 0.33243767]
 [0.42795455 0.49846514]
 [0.38893409 0.45101393]
 ...
 [0.14690934 0.20873368]
 [0.43249694 0.46400384]
 [0.28177512 0.17774063]]
